# 0. 환경설정

In [ ]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools
import copy

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

# 지수표현
pd.options.display.float_format = '{:.5f}'.format

# 마이너스 부호 깨짐 해결
import matplotlib
plt.rcParams['axes.unicode_minus'] = False

# # 그래프 한글폰트 해결
# from matplotlib import font_manager,rc
# font_location="malgun.ttf"
# font_name=font_manager.FontProperties(fname=font_location).get_name()
# rc('font',family=font_name)

-----------

# 1. 데이터 호출

In [ ]:
# # 컬럼 형식 확인
# df_nm = 'data_1'
# for i in globals()[df_nm].columns:
#     print(type(globals()[df_nm][i][0]))

In [ ]:
# 데이터 호출 함수
def load_data():

    # 예측값
    data_folder = 'pred'

    file_list = ['predict_result_AGE.csv',              # 시도별 학년
                 'predict_result_CLS_NM.csv',           # 시도별 시설유형
                 'predict_result_DIS_TYPE.csv',         # 시도별 장애영역
                 'predict_result_SCHOOL_TP.csv',        # 교육지원청 학교
                 'predict_result_part_DIS_TYPE.csv',    # 시도*교육청별 지적_자폐성 : predict_result_part_dis2DIS_TYPE
                 'predict_result_part_DIS_TYPE_2.csv']  # 시도*교육청별 지적_자폐성_발달지체 : predict_result_part_dis3DIS_TYPE

    for file_nm in file_list:
        globals()['pred_' + (file_nm.split('result_')[1].split('.')[0])] = pd.read_csv(data_folder + '/' + file_nm, encoding = 'cp949')
        #print('pred_' + (file_nm.split('result_')[1].split('.')[0]))

    # 실적값
    data_folder = 'real'

    file_list = ['merge_age.csv',
                 'merge_cls.csv',
                 'merge_dis.csv',
                 'merge_part_age.csv',
                 'merge_part_dis2.csv',
                 'merge_part_dis3.csv',
                 'merge_student.csv',
                 'merge_teacher.csv']

    for file_nm in file_list:
        globals()['real_' + (file_nm.split('merge_')[1].split('.')[0])] = pd.read_csv(data_folder + '/' + file_nm, encoding = 'cp949')
        #print('real_' + (file_nm.split('merge_')[1].split('.')[0]))
        
    return print('데이터 호출 완료')

-----------

# 2. dict 정의

In [ ]:
sido_to_sido = {'강원도': '강원', '경기도': '경기', '경상남도': '경남', '경상북도': '경북',
                '광주광역시': '광주', '대구광역시': '대구', '대전광역시': '대전', '부산광역시': '부산',
                '서울특별시': '서울', '세종특별자치시': '세종', '울산광역시': '울산', '인천광역시': '인천',
                '전라남도': '전남', '전라북도': '전북', '제주특별자치도': '제주', '충청남도': '충남', '충청북도': '충북'}

sido_to_eng = {'강원':'GANGWON', '울산':'ULSAN', '충북':'CHUNGBUK', '충남':'CHUNGNAM', '제주':'JEJU',
               '전북':'JEONBUK', '전남':'JEONNAM', '인천':'INCHEON', '서울':'SEOUL', '경기':'GYEONGGI',
               '부산':'BUSAN', '대전':'DAEJEON', '대구':'DAEGU', '광주':'GWANGJU', '경북':'GYEONGBUK',
               '경남':'GYEONGNAM', '세종':'SEJONG'}

eng_sido_list = ['GANGWON', 'ULSAN', 'CHUNGBUK', 'CHUNGNAM', 'JEJU',
                 'JEONBUK', 'JEONNAM', 'INCHEON', 'SEOUL', 'GYEONGGI',
                 'BUSAN', 'DAEJEON', 'DAEGU', 'GWANGJU', 'GYEONGBUK',
                 'GYEONGNAM', 'SEJONG']

age_to_eng = {'유3':'KINDER_3', '유4':'KINDER_4', '유5이상':'KINDER_5to7',
              '초1':'ELEMENT_1', '초2':'ELEMENT_2', '초3':'ELEMENT_3', '초4':'ELEMENT_4', '초5':'ELEMENT_5', '초6':'ELEMENT_6',
              '중1':'MIDDLE_1', '중2':'MIDDLE_2', '중3':'MIDDLE_3',
              '고1':'HIGH_1', '고2':'HIGH_2', '고3':'HIGH_3'}

range_age_to_eng = {'유치원':'KINDER', '초등학교':'ELEMENT', '중학교':'MIDDLE', '고등학교':'HIGH'}

eng_age_list = ['KINDER_3', 'KINDER_4', 'KINDER_5to7',
                'ELEMENT_1', 'ELEMENT_2', 'ELEMENT_3', 'ELEMENT_4', 'ELEMENT_5', 'ELEMENT_6',
                'MIDDLE_1', 'MIDDLE_2', 'MIDDLE_3',
                'HIGH_1', 'HIGH_2', 'HIGH_3']

cls_to_eng = {'일반학급' : 'GNRL_CLASS', '특수학교' : 'SPCL_SCHOL', '특수학급' : 'SPCL_CLASS'}

eng_cls_list = ['GNRL_CLASS','SPCL_SCHOL','SPCL_CLASS']

dis_to_eng = {'특수_건강':'health', '특수_발달지체':'baldaljiche', '특수_시각':'sigak',
              '특수_의사소통':'uisasotong', '특수_자폐성':'japyeseong', '특수_정서행동':'jeongseohaengdong',
              '특수_지적':'jijeog', '특수_지체':'jiche', '특수_청각':'cheonggak', '특수_학습':'hagseub'}

eng_dis_list = ['health', 'baldaljiche', 'sigak',
                'uisasotong', 'japyeseong', 'jeongseohaengdong',
                'jijeog', 'jiche', 'cheonggak', 'hagseub']

edu_offi_to_eng = {'강원도교육청':'GANGWON_EDU_OFFI', '경기도교육청':'GYEONGGI_EDU_OFFI', '경상남도교육청':'GYEONGNAM_EDU_OFFI',
                   '경상북도교육청':'GYEONGBUK_EDU_OFFI', '광주광역시교육청':'GWANGJU_EDU_OFFI', '대구광역시교육청':'DAEGU_EDU_OFFI',
                   '대전광역시교육청':'DAEJEON_EDU_OFFI', '부산광역시교육청':'BUSAN_EDU_OFFI', '서울특별시교육청':'SEOUL_EDU_OFFI',
                   '세종특별자치시교육청':'SEJONG_EDU_OFFI', '울산광역시교육청':'ULSAN_EDU_OFFI', '인천광역시교육청':'INCHEON_EDU_OFFI',
                   '전라남도교육청':'JEONNAM_EDU_OFFI', '전라북도교육청':'JEONBUK_EDU_OFFI', '제주특별자치도교육청':'JEJU_EDU_OFFI',
                   '충청남도교육청':'CHUNGNAM_EDU_OFFI', '충청북도교육청':'CHUNGBUK_EDU_OFFI'}

edu_supp_offi_to_eng = {'서부교육지원청':'seobu_EDU_SUPP_OFFI', '동부교육지원청':'dongbu_EDU_SUPP_OFFI', '남부교육지원청':'nambu_EDU_SUPP_OFFI',
                        '북부교육지원청':'bugbu_EDU_SUPP_OFFI', '고성교육지원청':'goseong_EDU_SUPP_OFFI', '영암교육지원청':'yeongam_EDU_SUPP_OFFI',
                        '고흥교육지원청':'goheung_EDU_SUPP_OFFI', '보성교육지원청':'boseong_EDU_SUPP_OFFI', '화순교육지원청':'hwasun_EDU_SUPP_OFFI',
                        '장흥교육지원청':'jangheung_EDU_SUPP_OFFI', '강진교육지원청':'gangjin_EDU_SUPP_OFFI', '해남교육지원청':'haenam_EDU_SUPP_OFFI',
                        '함평교육지원청':'hampyeong_EDU_SUPP_OFFI', '무안교육지원청':'muan_EDU_SUPP_OFFI', '곡성교육지원청':'gogseong_EDU_SUPP_OFFI',
                        '영광교육지원청':'yeonggwang_EDU_SUPP_OFFI', '장성교육지원청':'jangseong_EDU_SUPP_OFFI', '완도교육지원청':'wando_EDU_SUPP_OFFI',
                        '진도교육지원청':'jindo_EDU_SUPP_OFFI', '신안교육지원청':'sinan_EDU_SUPP_OFFI', '포항교육지원청':'pohang_EDU_SUPP_OFFI',
                        '구례교육지원청':'gulye_EDU_SUPP_OFFI', '광양교육지원청':'gwangyang_EDU_SUPP_OFFI', '담양교육지원청':'damyang_EDU_SUPP_OFFI',
                        '장수교육지원청':'jangsu_EDU_SUPP_OFFI', '익산교육지원청':'igsan_EDU_SUPP_OFFI', '정읍교육지원청':'jeongeub_EDU_SUPP_OFFI',
                        '남원교육지원청':'namwon_EDU_SUPP_OFFI', '김제교육지원청':'gimje_EDU_SUPP_OFFI', '완주교육지원청':'wanju_EDU_SUPP_OFFI',
                        '진안교육지원청':'jinan_EDU_SUPP_OFFI', '무주교육지원청':'muju_EDU_SUPP_OFFI', '임실교육지원청':'imsil_EDU_SUPP_OFFI',
                        '김천교육지원청':'gimcheon_EDU_SUPP_OFFI', '순창교육지원청':'sunchang_EDU_SUPP_OFFI', '고창교육지원청':'gochang_EDU_SUPP_OFFI',
                        '부안교육지원청':'buan_EDU_SUPP_OFFI', '목포교육지원청':'mogpo_EDU_SUPP_OFFI', '여수교육지원청':'yeosu_EDU_SUPP_OFFI',
                        '순천교육지원청':'suncheon_EDU_SUPP_OFFI', '나주교육지원청':'naju_EDU_SUPP_OFFI', '경주교육지원청':'gyeongju_EDU_SUPP_OFFI',
                        '구미교육지원청':'gumi_EDU_SUPP_OFFI', '안동교육지원청':'andong_EDU_SUPP_OFFI', '전주교육지원청':'jeonju_EDU_SUPP_OFFI',
                        '통영교육지원청':'tongyeong_EDU_SUPP_OFFI', '사천교육지원청':'sacheon_EDU_SUPP_OFFI', '김해교육지원청':'gimhae_EDU_SUPP_OFFI',
                        '밀양교육지원청':'milyang_EDU_SUPP_OFFI', '거제교육지원청':'geoje_EDU_SUPP_OFFI', '의령교육지원청':'uilyeong_EDU_SUPP_OFFI',
                        '함안교육지원청':'haman_EDU_SUPP_OFFI', '창녕교육지원청':'changnyeong_EDU_SUPP_OFFI', '양산교육지원청':'yangsan_EDU_SUPP_OFFI',
                        '남해교육지원청':'namhae_EDU_SUPP_OFFI', '하동교육지원청':'hadong_EDU_SUPP_OFFI', '산청교육지원청':'sancheong_EDU_SUPP_OFFI',
                        '함양교육지원청':'hamyang_EDU_SUPP_OFFI', '거창교육지원청':'geochang_EDU_SUPP_OFFI', '합천교육지원청':'habcheon_EDU_SUPP_OFFI',
                        '제주시교육지원청':'jejusi_EDU_SUPP_OFFI', '서귀포시교육지원청':'seogwiposi_EDU_SUPP_OFFI', '진주교육지원청':'jinju_EDU_SUPP_OFFI',
                        '창원교육지원청':'changwon_EDU_SUPP_OFFI', '울릉교육지원청':'ulleung_EDU_SUPP_OFFI', '청송교육지원청':'cheongsong_EDU_SUPP_OFFI',
                        '영주교육지원청':'yeongju_EDU_SUPP_OFFI', '영천교육지원청':'yeongcheon_EDU_SUPP_OFFI', '상주교육지원청':'sangju_EDU_SUPP_OFFI',
                        '문경교육지원청':'mungyeong_EDU_SUPP_OFFI', '경산교육지원청':'gyeongsan_EDU_SUPP_OFFI', '군위교육지원청':'gunwi_EDU_SUPP_OFFI',
                        '의성교육지원청':'uiseong_EDU_SUPP_OFFI', '영양교육지원청':'yeongyang_EDU_SUPP_OFFI', '울진교육지원청':'uljin_EDU_SUPP_OFFI',
                        '영덕교육지원청':'yeongdeog_EDU_SUPP_OFFI', '청도교육지원청':'cheongdo_EDU_SUPP_OFFI', '고령교육지원청':'golyeong_EDU_SUPP_OFFI',
                        '성주교육지원청':'seongju_EDU_SUPP_OFFI', '칠곡교육지원청':'chilgog_EDU_SUPP_OFFI', '예천교육지원청':'yecheon_EDU_SUPP_OFFI',
                        '봉화교육지원청':'bonghwa_EDU_SUPP_OFFI', '군산교육지원청':'gunsan_EDU_SUPP_OFFI', '태안교육지원청':'taean_EDU_SUPP_OFFI',
                        '당진교육지원청':'dangjin_EDU_SUPP_OFFI', '파주교육지원청':'paju_EDU_SUPP_OFFI', '안산교육지원청':'ansan_EDU_SUPP_OFFI',
                        '고양교육지원청':'goyang_EDU_SUPP_OFFI', '평택교육지원청':'pyeongtaeg_EDU_SUPP_OFFI', '군포의왕교육지원청':'gunpouiwang_EDU_SUPP_OFFI',
                        '구리남양주교육지원청':'gulinamyangju_EDU_SUPP_OFFI', '여주교육지원청':'yeoju_EDU_SUPP_OFFI', '화성오산교육지원청':'hwaseongosan_EDU_SUPP_OFFI',
                        '광주하남교육지원청':'gwangjuhanam_EDU_SUPP_OFFI', '광명교육지원청':'gwangmyeong_EDU_SUPP_OFFI', '연천교육지원청':'yeoncheon_EDU_SUPP_OFFI',
                        '포천교육지원청':'pocheon_EDU_SUPP_OFFI', '가평교육지원청':'gapyeong_EDU_SUPP_OFFI', '양평교육지원청':'yangpyeong_EDU_SUPP_OFFI',
                        '이천교육지원청':'icheon_EDU_SUPP_OFFI', '용인교육지원청':'yongin_EDU_SUPP_OFFI', '안성교육지원청':'anseong_EDU_SUPP_OFFI',
                        '동두천양주교육지원청':'dongducheonyangju_EDU_SUPP_OFFI', '부천교육지원청':'bucheon_EDU_SUPP_OFFI', '예산교육지원청':'yesan_EDU_SUPP_OFFI',
                        '동래교육지원청':'donglae_EDU_SUPP_OFFI', '중부교육지원청':'jungbu_EDU_SUPP_OFFI', '강서양천교육지원청':'gangseoyangcheon_EDU_SUPP_OFFI',
                        '강남서초교육지원청':'gangnamseocho_EDU_SUPP_OFFI', '성북강북교육지원청':'seongbuggangbug_EDU_SUPP_OFFI', '강동송파교육지원청':'gangdongsongpa_EDU_SUPP_OFFI',
                        '동작관악교육지원청':'dongjaggwanag_EDU_SUPP_OFFI', '성동광진교육지원청':'seongdonggwangjin_EDU_SUPP_OFFI', '해운대교육지원청':'haeundae_EDU_SUPP_OFFI',
                        '안양과천교육지원청':'anyanggwacheon_EDU_SUPP_OFFI', '달성교육지원청':'dalseong_EDU_SUPP_OFFI', '강화교육지원청':'ganghwa_EDU_SUPP_OFFI',
                        '강북교육지원청':'gangbug_EDU_SUPP_OFFI', '강남교육지원청':'gangnam_EDU_SUPP_OFFI', '수원교육지원청':'suwon_EDU_SUPP_OFFI',
                        '성남교육지원청':'seongnam_EDU_SUPP_OFFI', '의정부교육지원청':'uijeongbu_EDU_SUPP_OFFI', '김포교육지원청':'gimpo_EDU_SUPP_OFFI',
                        '시흥교육지원청':'siheung_EDU_SUPP_OFFI', '춘천교육지원청':'chuncheon_EDU_SUPP_OFFI', '보령교육지원청':'bolyeong_EDU_SUPP_OFFI',
                        '영동교육지원청':'yeongdong_EDU_SUPP_OFFI', '진천교육지원청':'jincheon_EDU_SUPP_OFFI', '괴산증평교육지원청':'goesanjeungpyeong_EDU_SUPP_OFFI',
                        '음성교육지원청':'eumseong_EDU_SUPP_OFFI', '단양교육지원청':'danyang_EDU_SUPP_OFFI', '천안교육지원청':'cheonan_EDU_SUPP_OFFI',
                        '공주교육지원청':'gongju_EDU_SUPP_OFFI', '아산교육지원청':'asan_EDU_SUPP_OFFI', '원주교육지원청':'wonju_EDU_SUPP_OFFI',
                        '서산교육지원청':'seosan_EDU_SUPP_OFFI', '금산교육지원청':'geumsan_EDU_SUPP_OFFI', '논산계룡교육지원청':'nonsangyelyong_EDU_SUPP_OFFI',
                        '부여교육지원청':'buyeo_EDU_SUPP_OFFI', '서천교육지원청':'seocheon_EDU_SUPP_OFFI', '청양교육지원청':'cheongyang_EDU_SUPP_OFFI',
                        '홍성교육지원청':'hongseong_EDU_SUPP_OFFI', '옥천교육지원청':'ogcheon_EDU_SUPP_OFFI', '보은교육지원청':'boeun_EDU_SUPP_OFFI',
                        '제천교육지원청':'jecheon_EDU_SUPP_OFFI', '충주교육지원청':'chungju_EDU_SUPP_OFFI', '강릉교육지원청':'gangleung_EDU_SUPP_OFFI',
                        '동해교육지원청':'donghae_EDU_SUPP_OFFI', '태백교육지원청':'taebaeg_EDU_SUPP_OFFI', '속초양양교육지원청':'sogchoyangyang_EDU_SUPP_OFFI',
                        '삼척교육지원청':'samcheog_EDU_SUPP_OFFI', '홍천교육지원청':'hongcheon_EDU_SUPP_OFFI', '횡성교육지원청':'hoengseong_EDU_SUPP_OFFI',
                        '영월교육지원청':'yeongwol_EDU_SUPP_OFFI', '평창교육지원청':'pyeongchang_EDU_SUPP_OFFI', '정선교육지원청':'jeongseon_EDU_SUPP_OFFI',
                        '철원교육지원청':'cheolwon_EDU_SUPP_OFFI', '화천교육지원청':'hwacheon_EDU_SUPP_OFFI', '양구교육지원청':'yanggu_EDU_SUPP_OFFI',
                        '인제교육지원청':'inje_EDU_SUPP_OFFI', '청주교육지원청':'cheongju_EDU_SUPP_OFFI', '세종특별자치시교육청':'sejongsi_EDU_SUPP_OFFI'}

-----------

# 3. 결과 그래프 생성

## 3-1. 특수학급(학교) 학생수 Vs. 교사수

In [ ]:
# 데이터 호출
load_data()

In [ ]:
# cls 예측값
tmp_pred_CLS_NM = pred_CLS_NM[['BASE_YY', 'SIDO', '특수학교', '특수학급']]
tmp_pred_CLS_NM['SIDO'] = tmp_pred_CLS_NM['SIDO'].replace(sido_to_sido)
tmp_pred_CLS_NM.rename(columns = {'특수학교':'특수학교_학생수(예측값)', '특수학급':'특수학급_학생수(예측값)'}, inplace=True)

# 특수교사 실적값
real_teacher['특수교사수'] = real_teacher['유_특수1정_CNT1'] + real_teacher['유_특수2정_CNT1'] + real_teacher['초_특수1정_CNT1'] + real_teacher['초_특수2정_CNT1'] + real_teacher['중고_특수1정_CNT1'] + real_teacher['중고_특수2정_CNT1']
tmp_real_teacher = real_teacher[['BASE_YY','CLASS_NM','SIDO_NM','특수교사수']]
real_teacher_1 = tmp_real_teacher.loc[tmp_real_teacher['CLASS_NM'] == '특수학급'].rename(columns = {'특수교사수':'특수학급_특수교사수(실적값)'})[['BASE_YY','SIDO_NM','특수학급_특수교사수(실적값)']]
real_teacher_2 = tmp_real_teacher.loc[tmp_real_teacher['CLASS_NM'] == '특수학교'].rename(columns = {'특수교사수':'특수학교_특수교사수(실적값)'})[['BASE_YY','SIDO_NM','특수학교_특수교사수(실적값)']]
tmp_real_teacher = pd.merge(real_teacher_1, real_teacher_2)
tmp_real_teacher.rename(columns = {'SIDO_NM':'SIDO'}, inplace=True)

# cls 예측값 / 특수교사 실적값 병합
real_pred_ds = pd.merge(tmp_pred_CLS_NM, tmp_real_teacher, how = 'outer')

# cls 실적값
tmp_real_cls = real_cls[['BASE_YY','SIDO','CLS_NM','SEP_CNT']]
real_cls_1 = tmp_real_cls[tmp_real_cls['CLS_NM'] == '특수학급'].rename(columns = {'SEP_CNT':'특수학급_학생수(실적값)'})[['BASE_YY','SIDO','특수학급_학생수(실적값)']]
real_cls_2 = tmp_real_cls[tmp_real_cls['CLS_NM'] == '특수학교'].rename(columns = {'SEP_CNT':'특수학교_학생수(실적값)'})[['BASE_YY','SIDO','특수학교_학생수(실적값)']]
tmp_real_cls = pd.merge(real_cls_1,real_cls_2)

real_pred_ds = pd.merge(real_pred_ds, tmp_real_cls, how = 'outer')
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO']).reset_index(drop=True)

specialClass = pd.DataFrame()
specialSchool = pd.DataFrame()

sido_list = list(real_pred_ds['SIDO'].value_counts().index)
    
for sido in sido_list:
    tmp1 = real_pred_ds.loc[real_pred_ds['SIDO'] == sido][['BASE_YY','SIDO','특수학급_학생수(실적값)','특수학급_학생수(예측값)','특수학급_특수교사수(실적값)']]
    specialClass = specialClass.append(tmp1)
    
    tmp2 = real_pred_ds.loc[real_pred_ds['SIDO'] == sido][['BASE_YY','SIDO','특수학교_학생수(실적값)','특수학교_학생수(예측값)','특수학교_특수교사수(실적값)']]
    specialSchool = specialSchool.append(tmp2)

specialClass.rename(columns = {'특수학급_학생수(실적값)':'stdnt_real','특수학급_학생수(예측값)':'stdnt_pred','특수학급_특수교사수(실적값)':'tchr_real'}, inplace=True)
specialSchool.rename(columns = {'특수학교_학생수(실적값)':'stdnt_real','특수학교_학생수(예측값)':'stdnt_pred','특수학교_특수교사수(실적값)':'tchr_real'}, inplace=True)

specialClass['SIDO'] = specialClass['SIDO'].replace(sido_to_eng)
specialSchool['SIDO'] = specialSchool['SIDO'].replace(sido_to_eng)

# 교사 수 예측값은 (학생 수 / 4)로 대체
specialClass['tchr_pred'] = specialClass['stdnt_pred']/4
specialSchool['tchr_pred'] = specialSchool['stdnt_pred']/4

In [ ]:
## 특수학급 그래프 저장
for sido in eng_sido_list:
    tmp = specialClass.loc[specialClass['SIDO'] == sido]
    plt.figure(figsize = (12,6))
    plt.cla()  # Clear the current axes
    plt.plot(tmp['BASE_YY'], tmp['stdnt_real'], lw = 3, color = 'tomato')
    plt.plot(tmp['BASE_YY'], tmp['stdnt_pred'], lw = 3, color = 'tomato', linestyle = '--')
    plt.plot(tmp['BASE_YY'], tmp['tchr_real'], lw = 3, color = 'indigo')
    plt.plot(tmp['BASE_YY'], tmp['tchr_pred'], lw = 3, color = 'indigo', linestyle = '--')
    plt.xlabel('YEAR')
    plt.xticks(list(tmp['BASE_YY']))
    plt.ylabel('VALUE')
    plt.legend(['stdnt_real','stdnt_pred','tchr_real','tchr_pred'], loc = 'upper right')
    plt.title(sido + ' (SpecialClass)')
    plt.savefig('graph_image/SpecialClass/' + sido + '_SpecialClass.png')

In [ ]:
## 특수학교 그래프 저장
for sido in eng_sido_list:
    tmp = specialSchool.loc[specialSchool['SIDO'] == sido]
    plt.figure(figsize = (12,6))
    plt.cla()  # Clear the current axes
    plt.plot(tmp['BASE_YY'], tmp['stdnt_real'], lw = 3, color = 'tomato')
    plt.plot(tmp['BASE_YY'], tmp['stdnt_pred'], lw = 3, color = 'tomato', linestyle = '--')
    plt.plot(tmp['BASE_YY'], tmp['tchr_real'], lw = 3, color = 'indigo')
    plt.plot(tmp['BASE_YY'], tmp['tchr_pred'], lw = 3, color = 'indigo', linestyle = '--')
    plt.xlabel('YEAR')
    plt.xticks(list(tmp['BASE_YY']))
    plt.ylabel('VALUE')
    plt.legend(['stdnt_real','stdnt_pred','tchr_real','tchr_pred'], loc = 'upper right')
    plt.title(sido + ' (SpecialSchool)')
    plt.savefig('graph_image/SpecialSchool/' + sido + '_SpecialSchool.png')

## 3-2. 시도*유/초/중/고

In [ ]:
# 데이터 호출
load_data()

In [ ]:
# age 예측값
pred_AGE['SIDO'] = pred_AGE['SIDO'].replace(sido_to_sido)

# age 실적값
real_age = real_age[['BASE_YY','SIDO','AGE','SEP_CNT']]

In [ ]:
# 데이터프레임 형식 맞춰주기(예측값 형식을 실적값 형식으로 변환)
form = list(pd.DataFrame(real_age['AGE'].value_counts()).reset_index()['index'])
pred_nujuk_df = pd.DataFrame()
for col in form:
    tmp = pred_AGE[['BASE_YY','SIDO',col]]
    tmp.rename(columns = {col:'SEP_CNT'}, inplace=True)
    tmp['AGE'] = col
    pred_nujuk_df = pred_nujuk_df.append(tmp)
pred_age = copy.deepcopy(pred_nujuk_df)

real_age.rename(columns = {'SEP_CNT':'real_val'}, inplace=True)
pred_age.rename(columns = {'SEP_CNT':'pred_val'}, inplace=True)

In [ ]:
# 실적값 / 예측값 병합
real_pred_ds = pd.merge(real_age, pred_age, how = 'outer')
real_pred_ds['SIDO'] = real_pred_ds['SIDO'].replace(sido_to_eng)
real_pred_ds['AGE'] = real_pred_ds['AGE'].replace(age_to_eng)
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO'])

In [ ]:
# 각 age를 컬럼화(age : KINDER_3, real_val : 63.00000, pred_val : NaN ===> KINDER_3_real : 63.00000, KINDER_3_pred : NaN) / real_pred_ds와 nujuk_tmp1 비교
nujuk_tmp1 = pd.DataFrame()
for sido in eng_sido_list:
    tmp = real_pred_ds.loc[real_pred_ds['SIDO'] == sido]
    tmp.reset_index(drop=True,inplace=True)
    for age in eng_age_list:
        tmp2 = tmp.loc[tmp['AGE'] == age]
        tmp2.reset_index(drop=True,inplace=True)
        tmp2.rename(columns = {'real_val':(age + '_real'),'pred_val':(age + '_pred')}, inplace=True)
        tmp2 = tmp2[['BASE_YY','SIDO',(age + '_real'),(age + '_pred')]]
        if age == eng_age_list[0]:
            nujuk_tmp2 = tmp2
        else:
            nujuk_tmp2 = pd.concat([nujuk_tmp2, tmp2[[(age + '_real'),(age + '_pred')]]], axis=1)
    nujuk_tmp1 = nujuk_tmp1.append(nujuk_tmp2)

In [ ]:
# 그래프화 할 때 묶어서 그릴 단위
graph_gubun = [['KINDER_3','KINDER_4','KINDER_5to7'],
               ['ELEMENT_1','ELEMENT_2','ELEMENT_3'],
               ['ELEMENT_4','ELEMENT_5','ELEMENT_6'],
               ['MIDDLE_1','MIDDLE_2','MIDDLE_3'],
               ['HIGH_1','HIGH_2','HIGH_3']]

In [ ]:
## 그래프 그리기
for sido in eng_sido_list:
    for grp_gubun in graph_gubun:
        # grp_col_list : grp_gubun의 값에 '_pred'와 '_real'을 붙인 목록 : ELEMENT_1_real / ELEMENT_1_pred
        grp_col_list = list(pd.Series(grp_gubun).map(lambda x : x + '_real')) + list(pd.Series(grp_gubun).map(lambda x : x + '_pred'))
        graph_df = nujuk_tmp1[['BASE_YY','SIDO'] + grp_col_list]
        graph_df = graph_df.loc[graph_df['SIDO'] == sido,]
        
        plt.figure(figsize = (12,6))
        plt.cla()  # Clear the current axes
        for grp_num in range(len(grp_col_list)):
            colors = ['tomato','orange','indigo','royalblue','darkslateblue',
                      'crimson','olivedrab','goldenrod','orangered','dimgray',
                      'olive','darkslategray','darkblue','mediumvioletred','teal',
                      'lime','purple','gold','maroon','forestgreen']
            colors = colors[0 : len(grp_gubun)]  # grp_gubun 개수만큼 색깔 가져오기
            if grp_num < len(grp_gubun):         # real인 경우 실선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, color = colors[grp_num % len(grp_gubun)])
            else:                                # pred인 경우 점선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, linestyle = '--', color = colors[grp_num % len(grp_gubun)])
        plt.xlabel('YEAR')
        plt.xticks(list(graph_df['BASE_YY']))
        plt.ylabel('VALUE')
        plt.legend(grp_col_list, loc = 'upper right')
        
        if len(list(pd.Series(grp_gubun).str.contains('ELEMENT_1').value_counts())) != 1:    # ELEMENT_1를 포함하는 경우 : 초등학교 저학년
            plt.title(sido + ' (' + grp_gubun[0].split('_')[0] + '_LOWER_GRADE)')
            plt.savefig('graph_image/sido_age/' + sido + '_' + grp_gubun[0].split('_')[0] + '_LOWER_GRADE.png')
        elif len(list(pd.Series(grp_gubun).str.contains('ELEMENT_4').value_counts())) != 1:  # ELEMENT_4를 포함하는 경우 : 초등학교 고학년
            plt.title(sido + ' (' + grp_gubun[0].split('_')[0] + '_HIGHER_GRADE)')
            plt.savefig('graph_image/sido_age/' + sido + '_' + grp_gubun[0].split('_')[0] + '_HIGHER_GRADE.png')
        else:
            plt.title(sido + ' (' + grp_gubun[0].split('_')[0] + ')')
            plt.savefig('graph_image/sido_age/' + sido + '_' + grp_gubun[0].split('_')[0] + '.png')

## 3-3. 시도*일반학급/특수학교/특수학급

In [ ]:
# 데이터 호출
load_data()

In [ ]:
# cls 예측값
pred_CLS_NM['SIDO'] = pred_CLS_NM['SIDO'].replace(sido_to_sido)

# cls 실적값
real_cls = real_cls[['BASE_YY','SIDO','CLS_NM','SEP_CNT']]

In [ ]:
# 데이터프레임 형식 맞춰주기(예측값 형식을 실적값 형식으로 변환)
form = list(pd.DataFrame(real_cls['CLS_NM'].value_counts()).reset_index()['index'])
pred_nujuk_df = pd.DataFrame()
for col in form:
    tmp = pred_CLS_NM[['BASE_YY','SIDO',col]]
    tmp.rename(columns = {col:'SEP_CNT'}, inplace=True)
    tmp['CLS_NM'] = col
    pred_nujuk_df = pred_nujuk_df.append(tmp)
pred_cls = copy.deepcopy(pred_nujuk_df)

real_cls.rename(columns = {'SEP_CNT':'real_val'}, inplace=True)
pred_cls.rename(columns = {'SEP_CNT':'pred_val'}, inplace=True)

In [ ]:
# 실적값 / 예측값 병합
real_pred_ds = pd.merge(real_cls, pred_cls, how = 'outer')
real_pred_ds['SIDO'] = real_pred_ds['SIDO'].replace(sido_to_eng)
real_pred_ds['CLS_NM'] = real_pred_ds['CLS_NM'].replace(cls_to_eng)
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO'])

In [ ]:
# 각 CLS를 컬럼화(CLS_NM : GNRL_CLASS, real_val : 787.00, pred_val : NaN ===> GNRL_CLASS_real : 787.00, GNRL_CLASS_pred : NaN) / real_pred_ds와 nujuk_tmp1 비교
nujuk_tmp1 = pd.DataFrame()
for sido in eng_sido_list:
    tmp = real_pred_ds.loc[real_pred_ds['SIDO'] == sido]
    tmp.reset_index(drop=True,inplace=True)
    for cls in eng_cls_list:
        tmp2 = tmp.loc[tmp['CLS_NM'] == cls]
        tmp2.reset_index(drop=True,inplace=True)
        tmp2.rename(columns = {'real_val':(cls + '_real'),'pred_val':(cls + '_pred')}, inplace=True)
        tmp2 = tmp2[['BASE_YY','SIDO',(cls + '_real'),(cls + '_pred')]]
        if cls == eng_cls_list[0]:
            nujuk_tmp2 = tmp2
        else:
            nujuk_tmp2 = pd.concat([nujuk_tmp2, tmp2[[(cls + '_real'),(cls + '_pred')]]], axis=1)
    nujuk_tmp1 = nujuk_tmp1.append(nujuk_tmp2)

In [ ]:
# 그래프화 할 때 묶어서 그릴 단위
graph_gubun = [['GNRL_CLASS', 'SPCL_SCHOL', 'SPCL_CLASS']]

In [ ]:
## 그래프 그리기
for sido in eng_sido_list:
    for grp_gubun in graph_gubun:
        # grp_col_list : grp_gubun의 값에 '_pred'와 '_real'을 붙인 목록 : GNRL_CLASS_real / GNRL_CLASS_pred
        grp_col_list = list(pd.Series(grp_gubun).map(lambda x : x + '_real')) + list(pd.Series(grp_gubun).map(lambda x : x + '_pred'))
        graph_df = nujuk_tmp1[['BASE_YY','SIDO'] + grp_col_list]
        graph_df = graph_df.loc[graph_df['SIDO'] == sido,]
        
        plt.figure(figsize = (12,6))
        plt.cla()  # Clear the current axes
        for grp_num in range(len(grp_col_list)):
            colors = ['tomato','orange','indigo','royalblue','darkslateblue',
                      'crimson','olivedrab','goldenrod','orangered','dimgray',
                      'olive','darkslategray','darkblue','mediumvioletred','teal',
                      'lime','purple','gold','maroon','forestgreen']
            colors = colors[0 : len(grp_gubun)]  # grp_gubun 개수만큼 색깔 가져오기
            if grp_num < len(grp_gubun):         # real인 경우 실선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, color = colors[grp_num % len(grp_gubun)])
            else:                                # pred인 경우 점선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, linestyle = '--', color = colors[grp_num % len(grp_gubun)])
        plt.xlabel('YEAR')
        plt.xticks(list(graph_df['BASE_YY']))
        plt.ylabel('VALUE')
        plt.legend(grp_col_list, loc = 'upper right')
        plt.title(sido)
        plt.savefig('graph_image/sido_cls/' + sido + '_(Class_School).png')

## 3-4. 시도*장애영역별

In [ ]:
# 데이터 호출
load_data()

In [ ]:
# dis 예측값
pred_DIS_TYPE['SIDO'] = pred_DIS_TYPE['SIDO'].replace(sido_to_sido)

# dis 실적값
real_dis = real_dis[['BASE_YY','SIDO','DIS_TYPE','SEP_CNT']]

In [ ]:
# 데이터프레임 형식 맞춰주기(예측값 형식을 실적값 형식으로 변환)
form = list(pd.DataFrame(real_dis['DIS_TYPE'].value_counts()).reset_index()['index'])
pred_nujuk_df = pd.DataFrame()
for col in form:
    tmp = pred_DIS_TYPE[['BASE_YY','SIDO',col]]
    tmp.rename(columns = {col:'SEP_CNT'}, inplace=True)
    tmp['DIS_TYPE'] = col
    pred_nujuk_df = pred_nujuk_df.append(tmp)
pred_dis = copy.deepcopy(pred_nujuk_df)

real_dis.rename(columns = {'SEP_CNT':'real_val'}, inplace=True)
pred_dis.rename(columns = {'SEP_CNT':'pred_val'}, inplace=True)

In [ ]:
# 실적값 / 예측값 병합
real_pred_ds = pd.merge(real_dis, pred_dis, how = 'outer')
real_pred_ds['SIDO'] = real_pred_ds['SIDO'].replace(sido_to_eng)
real_pred_ds['DIS_TYPE'] = real_pred_ds['DIS_TYPE'].replace(dis_to_eng)
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO'])

In [ ]:
# 각 dis를 컬럼화
nujuk_tmp1 = pd.DataFrame()
for sido in eng_sido_list:
    tmp = real_pred_ds.loc[real_pred_ds['SIDO'] == sido]
    tmp.reset_index(drop=True,inplace=True)
    for dis in eng_dis_list:
        tmp2 = tmp.loc[tmp['DIS_TYPE'] == dis]
        tmp2.reset_index(drop=True,inplace=True)
        tmp2.rename(columns = {'real_val':(dis + '_real'),'pred_val':(dis + '_pred')}, inplace=True)
        tmp2 = tmp2[['BASE_YY','SIDO',(dis + '_real'),(dis + '_pred')]]
        if dis == eng_dis_list[0]:
            nujuk_tmp2 = tmp2
        else:
            nujuk_tmp2 = pd.concat([nujuk_tmp2, tmp2[[(dis + '_real'),(dis + '_pred')]]], axis=1)
    nujuk_tmp1 = nujuk_tmp1.append(nujuk_tmp2)

In [ ]:
# 그래프화 할 때 묶어서 그릴 단위
graph_gubun = [['jijeog','japyeseong','baldaljiche','jiche'],
               ['sigak','cheonggak','health'],
               ['uisasotong','jeongseohaengdong','hagseub']]

In [ ]:
## 그래프 그리기
for sido in eng_sido_list:
    for grp_gubun in graph_gubun:
        # grp_col_list : grp_gubun의 값에 '_pred'와 '_real'을 붙인 목록 : ELEMENT_1_real / ELEMENT_1_pred
        grp_col_list = list(pd.Series(grp_gubun).map(lambda x : x + '_real')) + list(pd.Series(grp_gubun).map(lambda x : x + '_pred'))
        graph_df = nujuk_tmp1[['BASE_YY','SIDO'] + grp_col_list]
        graph_df = graph_df.loc[graph_df['SIDO'] == sido,]
        
        plt.figure(figsize = (12,6))
        plt.cla()  # Clear the current axes
        for grp_num in range(len(grp_col_list)):
            colors = ['tomato','orange','indigo','royalblue','darkslateblue',
                      'crimson','olivedrab','goldenrod','orangered','dimgray',
                      'olive','darkslategray','darkblue','mediumvioletred','teal',
                      'lime','purple','gold','maroon','forestgreen']
            colors = colors[0 : len(grp_gubun)]  # grp_gubun 개수만큼 색깔 가져오기
            if grp_num < len(grp_gubun):         # real인 경우 실선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, color = colors[grp_num % len(grp_gubun)])
            else:                                # pred인 경우 점선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, linestyle = '--', color = colors[grp_num % len(grp_gubun)])
        plt.xlabel('YEAR')
        plt.xticks(list(graph_df['BASE_YY']))
        plt.ylabel('VALUE')
        plt.legend(grp_col_list, loc = 'upper right')
        plt.title(sido + ' (' + ', '.join(grp_gubun) + ')')
        plt.savefig('graph_image/sido_dis/' + sido + '_' + '_'.join(grp_gubun) + '.png')

## 3-5. 시도 * 교육청별 * 유초중고

In [ ]:
# 데이터 호출
load_data()

In [ ]:
# 예측값
pred_SCHOOL_TP['SIDO_EDU_NM'] = pred_SCHOOL_TP['SIDO_EDU_NM'].replace(edu_offi_to_eng)
pred_SCHOOL_TP['PART_EDU_NM'] = pred_SCHOOL_TP['PART_EDU_NM'].replace(edu_supp_offi_to_eng)
pred_SCHOOL_TP['SIDO_PART'] = pred_SCHOOL_TP['SIDO_EDU_NM'] + '+' + pred_SCHOOL_TP['PART_EDU_NM']
pred_SCHOOL_TP = pred_SCHOOL_TP[['SIDO_PART','BASE_YY','유치원','초등학교','중학교','고등학교']]
pred_SCHOOL_TP['SIDO_PART'] = pred_SCHOOL_TP['SIDO_PART'].map(lambda x : x.upper())

# 실적값
real_part_age['SIDO_EDU_NM'] = real_part_age['SIDO_EDU_NM'].replace(edu_offi_to_eng)
real_part_age['PART_EDU_NM'] = real_part_age['PART_EDU_NM'].replace(edu_supp_offi_to_eng)
real_part_age['SIDO_PART'] = real_part_age['SIDO_EDU_NM'] + '+' + real_part_age['PART_EDU_NM']
real_part_age = real_part_age[['SIDO_PART','BASE_YY','SCHOOL_TP','SEP_CNT']]
real_part_age['SIDO_PART'] = real_part_age['SIDO_PART'].map(lambda x : x.upper())

In [ ]:
# 데이터프레임 형식 맞춰주기(예측값 형식을 실적값 형식으로 변환)
form = list(pd.DataFrame(real_part_age['SCHOOL_TP'].value_counts()).reset_index()['index'])
pred_nujuk_df = pd.DataFrame()
for col in form:
    tmp = pred_SCHOOL_TP[['BASE_YY','SIDO_PART',col]]
    tmp.rename(columns = {col:'SEP_CNT'}, inplace=True)
    tmp['SCHOOL_TP'] = col
    pred_nujuk_df = pred_nujuk_df.append(tmp)
pred_school_tp = copy.deepcopy(pred_nujuk_df)

real_part_age.rename(columns = {'SEP_CNT':'real_val'}, inplace=True)
pred_school_tp.rename(columns = {'SEP_CNT':'pred_val'}, inplace=True)

In [ ]:
# 실적값 / 예측값 병합
real_pred_ds = pd.merge(real_part_age, pred_school_tp, how = 'outer')
real_pred_ds['SCHOOL_TP'] = real_pred_ds['SCHOOL_TP'].replace(range_age_to_eng)
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO_PART'])

In [ ]:
eng_sido_part_list = list(pd.DataFrame(real_pred_ds['SIDO_PART'].value_counts()).reset_index()['index'])
eng_sido_part_list.sort()

eng_school_tp_list = list(pd.DataFrame(real_pred_ds['SCHOOL_TP'].value_counts()).reset_index()['index'])
eng_school_tp_list.sort()

# 각 school_tp를 컬럼화
nujuk_tmp1 = pd.DataFrame()
for sido_part in eng_sido_part_list:
    tmp = real_pred_ds.loc[real_pred_ds['SIDO_PART'] == sido_part]
    tmp.reset_index(drop=True,inplace=True)
    for school_tp in eng_school_tp_list:
        tmp2 = tmp.loc[tmp['SCHOOL_TP'] == school_tp]
        tmp2.reset_index(drop=True,inplace=True)
        tmp2.rename(columns = {'real_val':(school_tp + '_real'),'pred_val':(school_tp + '_pred')}, inplace=True)
        tmp2 = tmp2[['BASE_YY','SIDO_PART',(school_tp + '_real'),(school_tp + '_pred')]]
        if school_tp == eng_school_tp_list[0]:
            nujuk_tmp2 = tmp2
        else:
            nujuk_tmp2 = pd.concat([nujuk_tmp2, tmp2[[(school_tp + '_real'),(school_tp + '_pred')]]], axis=1)
    nujuk_tmp1 = nujuk_tmp1.append(nujuk_tmp2)

In [ ]:
# 그래프화 할 때 묶어서 그릴 단위
graph_gubun = [['KINDER', 'ELEMENT', 'MIDDLE', 'HIGH']]

In [ ]:
## 그래프 그리기
for sido in eng_sido_part_list:
    for grp_gubun in graph_gubun:
        # grp_col_list : grp_gubun의 값에 '_pred'와 '_real'을 붙인 목록 : ELEMENT_1_real / ELEMENT_1_pred
        grp_col_list = list(pd.Series(grp_gubun).map(lambda x : x + '_real')) + list(pd.Series(grp_gubun).map(lambda x : x + '_pred'))
        graph_df = nujuk_tmp1[['BASE_YY','SIDO_PART'] + grp_col_list]
        graph_df = graph_df.loc[graph_df['SIDO_PART'] == sido,]
        
        plt.figure(figsize = (12,6))
        plt.cla()  # Clear the current axes
        for grp_num in range(len(grp_col_list)):
            colors = ['tomato','orange','indigo','royalblue','darkslateblue',
                      'crimson','olivedrab','goldenrod','orangered','dimgray',
                      'olive','darkslategray','darkblue','mediumvioletred','teal',
                      'lime','purple','gold','maroon','forestgreen']
            colors = colors[0 : len(grp_gubun)]  # grp_gubun 개수만큼 색깔 가져오기
            if grp_num < len(grp_gubun):         # real인 경우 실선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, color = colors[grp_num % len(grp_gubun)])
            else:                                # pred인 경우 점선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, linestyle = '--', color = colors[grp_num % len(grp_gubun)])
        plt.xlabel('YEAR')
        plt.xticks(list(graph_df['BASE_YY']))
        plt.ylabel('VALUE')
        plt.legend(grp_col_list, loc = 'upper right')
        plt.title(sido.replace('_EDU_OFFI','').replace('_EDU_SUPP_OFFI','').replace('+',' + '))
        plt.savefig('graph_image/sido_edu_part_age/' + sido.replace('_EDU_OFFI','').replace('_EDU_SUPP_OFFI','').replace('+',' + ') + '.png')

## 3-6. 시도 * 교육청별 (지적_자폐성 + 지적_자폐성_발달지체)

In [ ]:
# 데이터 호출
load_data()

### 3-6-1. 시도 * 교육청별 지적_자폐성

In [ ]:
# 예측값
pred_part_DIS_TYPE.rename(columns = {'지적_자폐':'지적_자폐성'}, inplace=True)
pred_part_DIS_TYPE['SIDO_EDU_NM'] = pred_part_DIS_TYPE['SIDO_EDU_NM'].replace(edu_offi_to_eng)
pred_part_DIS_TYPE['PART_EDU_NM'] = pred_part_DIS_TYPE['PART_EDU_NM'].replace(edu_supp_offi_to_eng)
pred_part_DIS_TYPE['SIDO_PART'] = pred_part_DIS_TYPE['SIDO_EDU_NM'] + '+' + pred_part_DIS_TYPE['PART_EDU_NM']
pred_part_DIS_TYPE = pred_part_DIS_TYPE[['SIDO_PART','BASE_YY','지적_자폐성']]
pred_part_DIS_TYPE['SIDO_PART'] = pred_part_DIS_TYPE['SIDO_PART'].map(lambda x : x.upper())

# 실적값
real_part_dis2['SIDO_EDU_NM'] = real_part_dis2['SIDO_EDU_NM'].replace(edu_offi_to_eng)
real_part_dis2['PART_EDU_NM'] = real_part_dis2['PART_EDU_NM'].replace(edu_supp_offi_to_eng)
real_part_dis2['SIDO_PART'] = real_part_dis2['SIDO_EDU_NM'] + '+' + real_part_dis2['PART_EDU_NM']
real_part_dis2 = real_part_dis2[['SIDO_PART','BASE_YY','DIS_TYPE','SEP_CNT']]
real_part_dis2['SIDO_PART'] = real_part_dis2['SIDO_PART'].map(lambda x : x.upper())

In [ ]:
# 데이터프레임 형식 맞춰주기(예측값 형식을 실적값 형식으로 변환)
form = list(pd.DataFrame(real_part_dis2['DIS_TYPE'].value_counts()).reset_index()['index'])
pred_nujuk_df = pd.DataFrame()
for col in form:
    tmp = pred_part_DIS_TYPE[['BASE_YY','SIDO_PART',col]]
    tmp.rename(columns = {col:'SEP_CNT'}, inplace=True)
    tmp['DIS_TYPE'] = col
    pred_nujuk_df = pred_nujuk_df.append(tmp)
pred_part_DIS_TYPE = copy.deepcopy(pred_nujuk_df)

real_part_dis2.rename(columns = {'SEP_CNT':'real_val'}, inplace=True)
pred_part_DIS_TYPE.rename(columns = {'SEP_CNT':'pred_val'}, inplace=True)

In [ ]:
# 실적값 / 예측값 병합
real_pred_ds = pd.merge(real_part_dis2, pred_part_DIS_TYPE, how = 'outer')
real_pred_ds['DIS_TYPE'] = real_pred_ds['DIS_TYPE'].replace({'지적_자폐성':'jijeog_japyeseong'})
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO_PART'])

In [ ]:
eng_sido_part_list = list(pd.DataFrame(real_pred_ds['SIDO_PART'].value_counts()).reset_index()['index'])
eng_sido_part_list.sort()

eng_dis_type_list = list(pd.DataFrame(real_pred_ds['DIS_TYPE'].value_counts()).reset_index()['index'])

# 각 dis_type을 컬럼화
nujuk_tmp1 = pd.DataFrame()
for sido_part in eng_sido_part_list:
    tmp = real_pred_ds.loc[real_pred_ds['SIDO_PART'] == sido_part]
    tmp.reset_index(drop=True,inplace=True)
    for dis_type in eng_dis_type_list:
        tmp2 = tmp.loc[tmp['DIS_TYPE'] == dis_type]
        tmp2.reset_index(drop=True,inplace=True)
        tmp2.rename(columns = {'real_val':(dis_type + '_real'),'pred_val':(dis_type + '_pred')}, inplace=True)
        tmp2 = tmp2[['BASE_YY','SIDO_PART',(dis_type + '_real'),(dis_type + '_pred')]]
        if dis_type == eng_dis_type_list[0]:
            nujuk_tmp2 = tmp2
        else:
            nujuk_tmp2 = pd.concat([nujuk_tmp2, tmp2[[(dis_type + '_real'),(dis_type + '_pred')]]], axis=1)
    nujuk_tmp1 = nujuk_tmp1.append(nujuk_tmp2)
df_jijeog_japyeseong = copy.deepcopy(nujuk_tmp1)

### 3-6-2. 시도*교육청별 지적_자폐성_발달지체

In [ ]:
# 예측값
pred_part_DIS_TYPE_2.rename(columns = {'지적_자폐_발달지체':'지적_자폐성_발달지체'}, inplace=True)
pred_part_DIS_TYPE_2['SIDO_EDU_NM'] = pred_part_DIS_TYPE_2['SIDO_EDU_NM'].replace(edu_offi_to_eng)
pred_part_DIS_TYPE_2['PART_EDU_NM'] = pred_part_DIS_TYPE_2['PART_EDU_NM'].replace(edu_supp_offi_to_eng)
pred_part_DIS_TYPE_2['SIDO_PART'] = pred_part_DIS_TYPE_2['SIDO_EDU_NM'] + '+' + pred_part_DIS_TYPE_2['PART_EDU_NM']
pred_part_DIS_TYPE_2 = pred_part_DIS_TYPE_2[['SIDO_PART','BASE_YY','지적_자폐성_발달지체']]
pred_part_DIS_TYPE_2['SIDO_PART'] = pred_part_DIS_TYPE_2['SIDO_PART'].map(lambda x : x.upper())

# 실적값
real_part_dis3['SIDO_EDU_NM'] = real_part_dis3['SIDO_EDU_NM'].replace(edu_offi_to_eng)
real_part_dis3['PART_EDU_NM'] = real_part_dis3['PART_EDU_NM'].replace(edu_supp_offi_to_eng)
real_part_dis3['SIDO_PART'] = real_part_dis3['SIDO_EDU_NM'] + '+' + real_part_dis3['PART_EDU_NM']
real_part_dis3 = real_part_dis3[['SIDO_PART','BASE_YY','DIS_TYPE','SEP_CNT']]
real_part_dis3['SIDO_PART'] = real_part_dis3['SIDO_PART'].map(lambda x : x.upper())

In [ ]:
# 데이터프레임 형식 맞춰주기(예측값 형식을 실적값 형식으로 변환)
form = list(pd.DataFrame(real_part_dis3['DIS_TYPE'].value_counts()).reset_index()['index'])
pred_nujuk_df = pd.DataFrame()
for col in form:
    tmp = pred_part_DIS_TYPE_2[['BASE_YY','SIDO_PART',col]]
    tmp.rename(columns = {col:'SEP_CNT'}, inplace=True)
    tmp['DIS_TYPE'] = col
    pred_nujuk_df = pred_nujuk_df.append(tmp)
pred_part_DIS_TYPE_2 = copy.deepcopy(pred_nujuk_df)

real_part_dis3.rename(columns = {'SEP_CNT':'real_val'}, inplace=True)
pred_part_DIS_TYPE_2.rename(columns = {'SEP_CNT':'pred_val'}, inplace=True)

In [ ]:
# 실적값 / 예측값 병합
real_pred_ds = pd.merge(real_part_dis3, pred_part_DIS_TYPE_2, how = 'outer')
real_pred_ds['DIS_TYPE'] = real_pred_ds['DIS_TYPE'].replace({'지적_자폐성_발달지체':'jijeog_japyeseong_baldaljiche'})
real_pred_ds = real_pred_ds.sort_values(by = ['BASE_YY','SIDO_PART'])

In [ ]:
eng_sido_part_list = list(pd.DataFrame(real_pred_ds['SIDO_PART'].value_counts()).reset_index()['index'])
eng_sido_part_list.sort()

eng_dis_type_list = list(pd.DataFrame(real_pred_ds['DIS_TYPE'].value_counts()).reset_index()['index'])

# 각 dis_type을 컬럼화
nujuk_tmp1 = pd.DataFrame()
for sido_part in eng_sido_part_list:
    tmp = real_pred_ds.loc[real_pred_ds['SIDO_PART'] == sido_part]
    tmp.reset_index(drop=True,inplace=True)
    for dis_type in eng_dis_type_list:
        tmp2 = tmp.loc[tmp['DIS_TYPE'] == dis_type]
        tmp2.reset_index(drop=True,inplace=True)
        tmp2.rename(columns = {'real_val':(dis_type + '_real'),'pred_val':(dis_type + '_pred')}, inplace=True)
        tmp2 = tmp2[['BASE_YY','SIDO_PART',(dis_type + '_real'),(dis_type + '_pred')]]
        if dis_type == eng_dis_type_list[0]:
            nujuk_tmp2 = tmp2
        else:
            nujuk_tmp2 = pd.concat([nujuk_tmp2, tmp2[[(dis_type + '_real'),(dis_type + '_pred')]]], axis=1)
    nujuk_tmp1 = nujuk_tmp1.append(nujuk_tmp2)
df_jijeog_japyeseong_baldaljiche = copy.deepcopy(nujuk_tmp1)

### 3-6-3. 지적_자폐성 + 지적_자폐성_발달지체 통합

In [ ]:
df_merge_jjb = pd.merge(df_jijeog_japyeseong, df_jijeog_japyeseong_baldaljiche, how = 'outer')

In [ ]:
# 그래프화 할 때 묶어서 그릴 단위
graph_gubun = [['jijeog_japyeseong','jijeog_japyeseong_baldaljiche']]

In [ ]:
## 그래프 그리기
for sido in eng_sido_part_list:
    for grp_gubun in graph_gubun:
        # grp_col_list : grp_gubun의 값에 '_pred'와 '_real'을 붙인 목록 : ELEMENT_1_real / ELEMENT_1_pred
        grp_col_list = list(pd.Series(grp_gubun).map(lambda x : x + '_real')) + list(pd.Series(grp_gubun).map(lambda x : x + '_pred'))
        graph_df = df_merge_jjb[['BASE_YY','SIDO_PART'] + grp_col_list]
        graph_df = graph_df.loc[graph_df['SIDO_PART'] == sido,]
        
        plt.figure(figsize = (12,6))
        plt.cla()  # Clear the current axes
        for grp_num in range(len(grp_col_list)):
            colors = ['tomato','orange','indigo','royalblue','darkslateblue',
                      'crimson','olivedrab','goldenrod','orangered','dimgray',
                      'olive','darkslategray','darkblue','mediumvioletred','teal',
                      'lime','purple','gold','maroon','forestgreen']
            colors = colors[0 : len(grp_gubun)]  # grp_gubun 개수만큼 색깔 가져오기
            if grp_num < len(grp_gubun):         # real인 경우 실선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, color = colors[grp_num % len(grp_gubun)])
            else:                                # pred인 경우 점선 처리
                plt.plot(graph_df['BASE_YY'], graph_df[grp_col_list[grp_num]], lw = 2, linestyle = '--', color = colors[grp_num % len(grp_gubun)])
        plt.xlabel('YEAR')
        plt.xticks(list(graph_df['BASE_YY']))
        plt.ylabel('VALUE')
        plt.legend(grp_col_list, loc = 'upper right')
        plt.title(sido.replace('_EDU_OFFI','').replace('_EDU_SUPP_OFFI','').replace('+',' + '))
        plt.savefig('graph_image/sido_edu_part_jj_jjb/' + sido.replace('_EDU_OFFI','').replace('_EDU_SUPP_OFFI','').replace('+',' + ') + '.png')

-------